<font size="10">
    <div align="center">
        Técnicas y algoritmos de búsqueda
    </div>
</font>
<font size="6">
    <div align="center">
        Informe de la práctica 3
    </div>
    <br>

</font>
<font size="5">
    <div align="center">
        Gabriel Niculescu Ruso 
    </div>
    <div align="center">
        Carlos Molera Canals
    </div>
    <div align="center">
        Jose Francisco Pérez Mompeán
    </div>
</font>
<font size="5">
    <div align="right">
        29/05/2025
    </div>
</font>


# Index
* [<font size="5">Introduction</font>](#introduction)
* [<font size="5">Alpha-Beta</font>](#Implementation)
* [<font size="5">Tabla de transposición</font>](#Entropy)
* [<font size="5">Función de evaluación</font>](#Automated-Clustering)
* [<font size="5">La red</font>](#Image-Segmentation)
# Introduction
[⬆️ Back to top](#index)

## Tabla de Transposición

A pesar de las ventajas que la búsqueda Alpha-Beta presenta, es posible optimizar aún más el tiempo que esta transcurre así como el número de nodos que se visitan. Una técnica clásica muy utiliazada en esta situación son las Tablas de Transposición. Durante la búsqueda es bastante común, especialmente con árboles con un alto factor de ramificación, que se llegue a una posición que se ha evaluado anteriormente mediante sequencias de acciones distintas. Dichos estados se conocen como transposiciones y obviamente, dichas situaciones son desfavorables ya que estamos gastando tiempo en cálculos que ya hemos realizado previamente, y es por esto que existen las tablas de transposiciones.

En esencia, una tabla de transposición es un mapa que almacena posiciones y las asocia con la evaluación que se obtuvo en dicha posición. Sin embargo, hay varios puntos importantes a considerar sobre como se guardan las evaluaciones y como se recuperan:

### Tipos de nodos

Desde los comienzos de Alpha-Beta, se notaron las diferencias entre los nodos que se evalúan durante la búsqueda. Donald Knuth y Ronald Moore fueron los primeros en nombrar estos nodos (Knuth Type 1, Knuth Type 2, Knuth Type 3). Mediante los análisis de varios otros investigadores como Judea Pearl, Tony Marsland y Fred Popowich, se les cambió a denominar *PV-Nodes*, *Cut-Nodes* y *All-Nodes*. Estos tipos de nodos determinan el significado de la evaluación guardada en la tabla, por lo que se explicará el significado de cada uno:

<u>***PV-Nodes***</u>

Los nodos de este tipo tienen como característica que la evaluación *s* obtenida por un nodo cae entre $\alpha$ y $\beta$, de manera que es el resultado exacto de realizar la búsqueda en esa rama. Además, dado esta característica, es el mismo mejor resultado que se obtendría de haber realizado una búsqueda minimax completa. Debido a lo mencionado, podemos decir que el valor de la evaluación obtenido es exacto y no un cota superior o inferior de su valor real.

Como información adicional, el nombre *PV-node* viene de una variación de la búsqueda Alpha-Beta conocida como **Principal Variation Search** (*PV Search*), donde los únicos nodos que se tratan de expandir son aquellos con el mismo nombre. En nuestro caso no implementamos esta optimización, sin embargo sería un esfuerzo interesante estudiarlo en el futuro.

<u>***Cut-Nodes***</u>

Estos nodos se dan cuando la evaluación obtenida ***s*** es mayor que $\beta$, por lo que se produce un *beta-cutoff*. Esto quiere decir que ***s*** es **tan buena** que el oponente va a evitar que realicemos ese movimiento. Debido a esto, no tiene sentido continuar con la búsqueda en de los hijos de este nodo, por lo que se corta la rama y se abandona, dando el nombre a este tipo de nodo.

Dado que no hemos evaluado todos los nodos hijos, no sabemos la evaluación exacta ***S*** del nodo padre, por lo que no podemos decir con certeza que la evaluación obtenida hasta ahora ***s*** es la resultante de haber realizado minimax. Debido ha esto, tenemos que la evaluación ***s*** es una cota de ***S***. Particularmente, como no conocemos todas las evaluaciones, decimos que es la **cota inferior** de ***S***, ya que esta debe ser al menos tan buena como $\beta$.

<u>***All-Nodes***</u>

Este tipo de nodos ocurren cuando se han explorado todos los hijos de un nodo particular y la mejor evaluación ***s*** es menor o igual que $\alpha$. Esto significa que ***s*** es peor o igual a la mejor evaluación actual, por lo que ninguna de las acciones exploradas es deseable en comparación con otras ya exploradas anteriormente. Debido a esto, podemos decir que ***s*** es la **cota superior** de la mejor evaluación dada por $\alpha$, ya que sabemos que la mejor evaluación es menor o igual a este valor.

Como nota adicional, este tipo de nodos son aquellos que más ralentizan el proceso de búsqueda dentro de Alpha-Beta. Dado que se deben explorar todos los hijos para determinar si alguna evaluación es mayor que $\alpha$, pero ninguno resulta serlo, este tipo de nodos representan un gran coste computacional. Es por eso que hay técnicas que ordenan los movimiento antes de extender la búsqueda de manera que los mejores movimientos se visiten primero. Técnicas como **Heuristic Move Ordering** o **Iterative Deepening** permiten encontrar mejores movimientos antes en la búsqueda, lo que permite saltarse más nodos de este tipo.

### Método de almacenamiento

Con lo expuesto, somos capaces de categorizar nodos de forma acorde a su evaluación. Sin embargo, aún no hemos discutido de que manera codificamos un estado particular, por lo que a continuación exponemos como lo hacemos y los problemas con los que nos encontramos.

Como se discutió previamente, las tablas de transposición asocian estados de la partida a evaluaciones y otra información útil. Esta asociación normalmente viene dada por un mapa (o diccionario en Python), donde se especifícan las llaves como la codificación de las partidas y los valores como las evaluaciones de estas. El método de codificación, sin embargo, difiere entre aplicaciones debido a la naturaleza de los estados mismos.

Por ejemplo, en ajedrez, dos métodos comunes para codificar el estado de un tablero son **Zobrist Hashing** y **BCH Hashing** (*Bose-Chaudhuri-Hocquenghem*). Estos métodos incluyen técnicas muy por encima de lo necesario para este proyecto, y más relevante, muy particulares para el juego del ajedrez. Implementar las técnicas que usan no es *imposible*, pero sería una tarea que habría tomado más tiempo del presente para esta práctica.

En su lugar, el código proporcionado por *UC Berkeley* hace uso de la función estándar *hash* de Python. Mediante esta función, se codifica toda la información posible sobre un estado en un solo número, de manera que podemos utilizarla para almancenar los estados del juego en la tabla de transposición.

<div align=center>
    <img src="../fotos/transposition_hash.png"/>
</div>

El código mostrado es el método mencionado anteriormente. Como se puede observar, hace uso de la información de la que se compone un estado de la partida, como los estados de pacman y los fantasmas, la comida, lás cápsulas y la puntuación actual. Sin embargo, este código incluye un problema muy grave, este siendo el módulo que se encuentra al final.

Después de extensiva búsqueda, encontramos que no hay razón aparente para la que exista este módulo. Ni en el [repositorio original](https://github.com/karlapalem/UC-Berkeley-AI-Pacman-Project) ni en la [página del projecto](https://inst.eecs.berkeley.edu/~cs188/fa24/projects/) encontramos referencia a esta decisión. Y no es sin razón que encontramos que es una decisión errónea:

1. El módulo limita el número de estados posibles a $1048575$, sin embargo esto no depende del tamaño del mapa
2. Estados diferentes son capaces de acabar con el mismo hash debido al efecto del módulo
3. Es posible (aunque raro) que las operaciones matemáticas resulten en el mismo hash, debido a que no se ha tenido cuidado en como se han realizado

En la práctica, encontramos que el uso de la tabla de transpoción resultaba en distintas partidas, cuando su uso no debería afectar a los resultados que la búsqueda Alpha-Beta habría obtenido. Tras un análisis riguroso de los hashes usados y las situaciones precisas en las que las diferencias se originaban, encontramos que el módulo causaba que estados con hashes distintos resultaran en la misma codificación.

Este problema está relacionado con uno mucho más amplio, conocido como *Hash Collision*. Este problema surge por el hecho de que los hashes intentan comprimir un número infinito de elementos dentro de un conjunto finito de codificaciones. Debido a esto, no es posible que un objeto o elemento particular tenga un hash totalmente único, sin embargo, es gracias al gran ingenio de los desarrolladores de estos hashes que se previene que este problema ocurra frequentemente, aunque nunca sea posible evitarlos del todo.

En comparación, el hash dado por el código original restringe el número de estados sin explición alguna, y es evidente que hay mapas que poseen más de los estados mencionados. Debido a esto, encontrabamos que algunos estados obtenían las evaluaciones de estados diferentes, lo cual es un gra problema para una técnica como la tabla de transposición. 

Para solucionar estos problemas, optamos por una técnica ligeramente distinta de guardar la información de un estado, que se puede observar en la siguiente imagen. Aquí, dejamos de lado el módulo anterior y usamos los resultados de los hashes en sí mismos. Dado que los números generados por estos son enormes (fácilmente en los trillones), es mucho menos común que hayan colisiones, aunque las operaciones aritméticas intermedias que realizamos aumentan esta probabilidad ligeramente.

<div align=center>
    <img src="../fotos/transposition_new_hash.png"/>
</div>

Gracias a esto, encontramos que no se producen colisiones y los juegos dados por Alpha-Beta son idénticos a aquellos dados por la Tabla de Transposición bajo las mismas situaciones. En nuestras pruebas, no hemos detectado ningún caso en el que ambos den lugar a juegos distintos aunque probásemos cientos de juegos, por lo que podemos concluir que este método de codificación es lo suficientemente robusto.

### Lectura

<div align=center>
    <img src="../fotos/transposition_lookup.png"/>
</div>

Para leer de la Tabla de Transposición, le proporcionamos la codificación del estado al cual queremos acceder, la profundidad de la búsqueda actual y los valores de $\alpha$ y $\beta$. En primer lugar, es posible que el estado actual no se haya visitado anteriormente, por lo que el estado no se encontrará en la tabla e informaremos de que ha ocurrido un error buscando el estado.

De no ser así, obtendremos el objeto entrada descrito posteriormente, que almacena toda la información importante sobre el estado actual. Sin embargo, no podemos hacer uso de dicha información incondicionalmente, algunas condiciones deben cumplirse. Primeramente, solo haremos uso de la evaluación almacenada si la profundidad a la que se encontró dicha evaluación es al menos tan profunda como la del estado anterior, sino más. Esto se debe a que queremos hacer uso de aquellas evaluaciones que hayan considerado más nodos en su búsqueda, ya que estas incluirán mayor cantidad de información sobre las posibles respuestas del oponente.

En segundo lugar, debemos considerar el tipo de evaluación que se dió cuando se guardó esta anteriormente. El hecho de que se haya dado el mismo estado en un momento previo de la búsqueda no implica que su tipo de evaluación sea el mismo, por lo que debemos concretar si se dan las mismas condiciones antes de hacer uso de la evaluación. Como se observa en la imágen, las comprobaciones que se realizan son las mismas que se usan para determinar el tipo de evaluación, por lo que solo si las condiciones del estado actual son las mismas que se dieron cuando se almacenó anteriormente haremos uso de lo guardado.

<div align=center>
    <img src="../fotos/transposition_entry.png"/>
</div>

## La Red